In [14]:
import numpy as np
class Transportation_Method(object):
    def __init__(self,supply,demand,cost):
        self.supply=supply
        self.cost=cost
        self.demand=demand
        self.transport_matrix=np.zeros((supply.size,demand.size))
    def northwestcorner_rule(self):
        i=0
        j=0
        supply_temp,demand_temp,transportmatrix_temp,cost_temp=self.__surplus()
        while i<demand_temp.size and j<supply_temp.size:
            if demand_temp[i]>=supply_temp[j]:
                amount=supply_temp[j]
                #print("Demand is bigger", amount)
            else:
                amount=demand_temp[i]
                #print("Supply is bigger", amount)
            demand_temp[i]=demand_temp[i]-amount
            supply_temp[j]=supply_temp[j]-amount
            transportmatrix_temp[j][i]=amount
            if supply_temp[j]==0:
                j+=1
            else:
                i+=1
        total_costs,surplus=self.__costs(transportmatrix_temp)
        return transportmatrix_temp,total_costs,surplus
    def colmin_rule(self):
        infinity=np.inf
        supply_temp,demand_temp,transportmatrix_temp,cost_temp=self.__surplus()
        cols=list(range(0,demand_temp.size))
        rows=list(range(0,supply_temp.size))
        while len(cols)>0:
            i=min(cols)
            temp=cost_temp[:, i]
            temp=temp.tolist()
            minima=min(temp)
            j=temp.index(minima)
            while j not in rows:
                if self.surplus=="demand" and len(rows)==1:
                    j=supply_temp.size-1
                    break
                temp[j]=infinity
                minima=min(temp)
                j=temp.index(minima)
                if not rows:
                    break
            if demand_temp[i]>=supply_temp[j]:
                amount=supply_temp[j]
                #print("Demand is bigger", amount)
            else:
                amount=demand_temp[i]
                #print("Supply  is bigger", amount)
            demand_temp[i]=demand_temp[i]-amount
            supply_temp[j]=supply_temp[j]-amount
            transportmatrix_temp[j][i]=amount
            if supply_temp[j]==0 or (np.sum(demand_temp)==0 and supply_temp[j]==infinity):
                rows.remove(j)
            if demand_temp[i]==0 or (np.sum(supply_temp)==0 and demand_temp[i]==infinity):
                cols.remove(i)
        total_costs,surplus=self.__costs(transportmatrix_temp)
        return transportmatrix_temp,total_costs,surplus
    def __surplus(self):
        infinity=np.inf
        supply_temp=np.copy(self.supply)
        demand_temp=np.copy(self.demand)
        transportmatrix_temp=np.copy(self.transport_matrix)
        cost_temp=np.copy(self.cost)
        if np.sum(supply_temp)>np.sum(demand_temp):
            demand_temp=np.append(demand_temp,infinity)
            new_column_temp=np.zeros((supply_temp.size,1))
            transportmatrix_temp=np.append(transportmatrix_temp,new_column_temp,axis=1)
            cost_temp=np.append(cost_temp,new_column_temp,axis=1)
            self.surplus="supply"
        elif np.sum(supply_temp)<np.sum(demand_temp):
            supply_temp=np.append(supply_temp,infinity)
            new_row_t_temp=np.zeros((1,demand_temp.size))
            new_row_c_temp=np.full((1,demand_temp.size),infinity)
            transportmatrix_temp=np.append(transportmatrix_temp,new_row_t_temp,axis=0)
            cost_temp=np.append(cost_temp,new_row_c_temp,axis=0)
            self.surplus="demand"
        else:
            self.surplus="equal"
        return supply_temp,demand_temp,transportmatrix_temp,cost_temp
    def __costs(self,transport_matrix):
        surplus = 0
        transport_matrix=np.copy(transport_matrix)
        if self.surplus=="demand":
            surplus=np.sum(transport_matrix[-1,:])
            surplus=surplus*-1
            transport_matrix=np.delete(transport_matrix,-1,axis=0)
        if self.surplus=="supply":
            surplus=np.sum(transport_matrix[:,-1])
            transport_matrix=np.delete(transport_matrix,-1,axis=1)
        temp=np.multiply(transport_matrix,self.cost)
        return temp.sum(),surplus
    
    


In [16]:
def calc():
    supply=np.array([5000,6000,2500])
    demand=np.array([6000,4000,2000,1500])
    cost=np.array([[3,2,7,6],
                   [7,5,2,3],
                   [2,5,4,5]])
    question=Transportation_Method(supply,demand,cost)
    print(".......COLUMN MINIMA RULE......")
    matrix,costs,surplus=question.colmin_rule()
    print("THE MATRIX IS:\n ",matrix)
    print("THE TOTAL COST IS: ",costs)
    print("UNITS OF SURPLUS: ",surplus)
    print(".......NORTH WEST CORNER RULE.........")
    matrix,costs,surplus=question.northwestcorner_rule()
    print("Matrix: \n",matrix)
    print("Total costs: ",costs)
    print("Surplus in units: ",surplus)
    

calc()

.......COLUMN MINIMA RULE......
THE MATRIX IS:
  [[3500. 1500.    0.    0.]
 [   0. 2500. 2000. 1500.]
 [2500.    0.    0.    0.]]
THE TOTAL COST IS:  39500.0
UNITS OF SURPLUS:  0
.......NORTH WEST CORNER RULE.........
Matrix: 
 [[5000.    0.    0.    0.]
 [1000. 4000. 1000.    0.]
 [   0.    0. 1000. 1500.]]
Total costs:  55500.0
Surplus in units:  0
